# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from scipy import stats as st

## Загрузка и подготовка данных

In [2]:
data_0 = pd.read_csv('geo_data_0.csv')
data_1 = pd.read_csv('geo_data_1.csv')
data_2 = pd.read_csv('geo_data_2.csv')

**Первый регион**

In [3]:
display(data_0.head())
data_0.info()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Второй регион**

In [4]:
display(data_1.head())
data_1.info()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


**Третий регион**

In [5]:
display(data_2.head())
data_2.info()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Согласно документации к данным:

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

Из исходных наборов данных удалим параметр `id`, т.к. в нём нет необходимости для исследования:

In [6]:
data_0 = data_0.drop('id', axis=1)
data_1 = data_1.drop('id', axis=1)
data_2 = data_2.drop('id', axis=1)

Разделим данные на признаки и целевой признак:

In [7]:
features_0 = data_0.drop('product', axis=1)
features_1 = data_1.drop('product', axis=1)
features_2 = data_2.drop('product', axis=1)

target_0 = data_0['product'] 
target_1 = data_1['product'] 
target_2 = data_2['product'] 

Разделение на обучающую и валидационную выборки:
- обучающая выборка – 75% 
- валидационная выборка – 25%

In [8]:
# Первый регион
features_train_0, features_valid_0, target_train_0, target_valid_0 = train_test_split(
                                        features_0, target_0, test_size=0.25, random_state=12345)
# Второй регион
features_train_1, features_valid_1, target_train_1, target_valid_1 = train_test_split(
                                        features_1, target_1, test_size=0.25, random_state=12345)
# Третий регион
features_train_2, features_valid_2, target_train_2, target_valid_2 = train_test_split(
                                        features_2, target_2, test_size=0.25, random_state=12345)

Размеры полученных выборок:

In [9]:
print('Первый регион', '\nОбучающая:', features_train_0.shape, '\nВалидационная:', features_valid_0.shape)
print('\nВторой регион', '\nОбучающая:', features_train_1.shape, '\nВалидационная:', features_valid_1.shape)
print('\nТретий регион', '\nОбучающая:', features_train_2.shape, '\nВалидационная:', features_valid_2.shape)

Первый регион 
Обучающая: (75000, 3) 
Валидационная: (25000, 3)

Второй регион 
Обучающая: (75000, 3) 
Валидационная: (25000, 3)

Третий регион 
Обучающая: (75000, 3) 
Валидационная: (25000, 3)


## Обучение и проверка модели

In [10]:
def fit_model(features_train, target_train, features_valid, target_valid):
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    MSE = mean_squared_error(target_valid, predicted_valid)
    return np.sqrt(MSE), predicted_valid

**Обучение модели на данных для первого региона:**

In [11]:
rmse_0, predicted_valid_0 = fit_model(features_train_0, target_train_0, features_valid_0, target_valid_0)
print(f'RMSE: {rmse_0}, \nCредний запас предсказанного сырья: {predicted_valid_0.mean()}')

RMSE: 37.5794217150813, 
Cредний запас предсказанного сырья: 92.59256778438035


**Обучение модели на данных для второго региона:**

In [12]:
rmse_1, predicted_valid_1 = fit_model(features_train_1, target_train_1, features_valid_1, target_valid_1)
print(f'RMSE: {rmse_1}, \nCредний запас предсказанного сырья: {predicted_valid_1.mean()}')

RMSE: 0.8930992867756165, 
Cредний запас предсказанного сырья: 68.728546895446


**Обучение модели на данных для третьего региона:**

In [13]:
rmse_2, predicted_valid_2 = fit_model(features_train_2, target_train_2, features_valid_2, target_valid_2)
print(f'RMSE: {rmse_2}, \nCредний запас предсказанного сырья: {predicted_valid_2.mean()}')

RMSE: 40.02970873393434, 
Cредний запас предсказанного сырья: 94.96504596800492


**Вывод**

Для оценки работы модели была рассчитана метрика RMSE: наивысшее качество модель показала на данных второго региона.
Наибольший средний запас предсказанного сырья в треьем регионе, а наименьший во втором.

## Подготовка к расчёту прибыли

Ключевые значения для расчётов 

In [14]:
# доход с одного барреля (р.)
REVENUE = 450

# Бюджет на разработку скважин в регионе
COST = 10**10

# Количество скважин
COUNT_WELLS = 200

# Достаточный объём сырья для безубыточной разработки новой скважины
MATERIAL = COST / 450000 / COUNT_WELLS
print("Достаточный объём сырья (тыс.бар.) для безубыточной разработки новой скважины:", MATERIAL)

Достаточный объём сырья (тыс.бар.) для безубыточной разработки новой скважины: 111.11111111111111


Сравнение среднего запаса сырья в каждом регионе с точкой безубыточности:

In [15]:
pd.Series([target_0.mean(), target_1.mean(), target_2.mean(), MATERIAL], index=['первый регион', 'второй регион', 'третий регоин', 'точка безубыточности'])

первый регион            92.500000
второй регион            68.825000
третий регоин            95.000000
точка безубыточности    111.111111
dtype: float64

**Вывод**

Во всех трех регионах средний обьем запасов в скважинах ниже достаточного объема сырья для безубыточной разработки новой скважины.

## Расчёт прибыли и рисков 

Функция для расчёта прибыли по выбранным скважинам и предсказаниям модели:

In [16]:
def payment_profit(target, predictions, count):
# выбрать скважины с максимальными значениями предсказаний
# просуммировать целевое значение объёма сырья, соответствующее этим предсказаниям
# рассчитать прибыль
    probs_sorted = predictions.sort_values(ascending=False)
    selected = target[probs_sorted.index][:count]
    return 450000 * selected.sum() - COST

Даллее найдем распределение прибыли с помощью техники bootstrap:

In [17]:
predictions = [predicted_valid_0, predicted_valid_1, predicted_valid_2]
targets = [target_valid_0,  target_valid_1, target_valid_2]

state = np.random.RandomState(12345)
profit_values = []
values = []
for reg in range(len(predictions)):
    predicted_valid = pd.Series(predictions[reg])
    target_valid = targets[reg].reset_index(drop=True)
    for i in range(1000):
        target_subsample = target_valid.sample(n=500, replace=True, random_state=state)
        probs_subsample = predicted_valid[target_subsample.index]
        profit_values.append(payment_profit(target_subsample, probs_subsample, COUNT_WELLS))
        
    profit_values = pd.Series(profit_values)
    mean = profit_values.mean()

    # Риск убытков
    losses = (profit_values < 0).mean()*100
    
    # 95%-й доверительный интервал
    lower = profit_values.quantile(0.025)
    upper = profit_values.quantile(0.975)
    
    print('Cредняя выручка:', mean)
    print('95%-ый доверительный интервал:', lower, ',', upper)
    print('Риск убытков:', losses)
    print()
    profit_values = []

Cредняя выручка: 425938526.91059244
95%-ый доверительный интервал: -102090094.83793654 , 947976353.3583689
Риск убытков: 6.0

Cредняя выручка: 518259493.6973249
95%-ый доверительный интервал: 128123231.43308444 , 953612982.0669085
Риск убытков: 0.3

Cредняя выручка: 420194005.3440501
95%-ый доверительный интервал: -115852609.16001143 , 989629939.8445739
Риск убытков: 6.2



**Вывод**

Наименьшие риски и наибольшая выручка во втором регионе.

## Общий вывод

Было проведено исследование скважин нефти в трех регионах и выявлено, что добыча нефти принесёт наибольшию прибыль, с наименьшими рисками – 1.3%, во втором регионе. С вероятностью 95% прибыль составит от 128'123231 до 953'612982.
